In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Exploração e Analise de Credito em SQL**

*Iremos analisar informações de credito de usuarios de um determinado banco*

## **Dados**:

Nesta base de dados contamos com as informaçoes do cliente do banco com as seguintes colunas :

* idade = idade do cliente
* sexo = sexo do cliente (F ou M)
* dependentes = número de dependentes do cliente
* escolaridade = nível de escolaridade do clientes
* estado_civil = estado civil do cliente
* salario_anual = faixa salarial do cliente
* tipo_cartao = tipo de cartao do cliente
* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
* iteracoes_12m = quantidade de iterações/transacoes nos ultimos 12 meses
* meses_inativo_12m = quantidade de meses que o cliente ficou inativo
* limite_credito = limite de credito do cliente
* valor_transacoes_12m = valor das transações dos ultimos 12 meses
* qtd_transacoes_12m  = quantidade de transacoes dos ultimos 12 meses

A tabela foi criada com a ajuda do AWS Athesa junto com a base de dados no S3 Bucket com os dados disponibilizados em :https://github.com/andre-marcos-perez/ebac-course-utils/tree/main/dataset

## **Exploração:** 
Primeiramente iremos analisar o que temos de materia-prima:

**Quantidade total de itens na base de dados?**
*Query:* Select count() from credito;
> Resultado: 2564 linhas

**ps:** Nota: Utilizamos apenas parte do dataset com intuito educacional, para facilitar a análise e evitar custos adicionais. Caso queira trabalhar com o dataset completo, o link está disponibilizado acima.

**Vamos verificar como fica a visualização da tabela** (*Iremos verificar somente as 08 primeiras linhas*):
*Query:* Select * from credito limit 8;

![Primeiras_Linhas](https://github.com/oGuiCorreia/EbacSql-/blob/main/primeiras_linhas.png?raw=true)

>Com esta analise também podemos perceber que alguns campos possuem valores nulos (na)

**E qual seria o valor aceito em cada coluna na tabela ?**
*Query:* Describe credito;

![Descricao_Tabela](https://github.com/oGuiCorreia/EbacSql-/blob/main/DescricaoDados.png?raw=true)

**Tipos de escolaridades disponiveis no dataset**
*Query:* Select distinct escolaridade from credito;

![Escolaridade](https://github.com/oGuiCorreia/EbacSql-/blob/main/Escolaridade.png?raw=true)

> Há variedade nos níveis de escolaridade cadastrados, mas também há valores nulos (NA).)

**Tipos de cartões**

*Query:* Select distinct tipo_cartao from credito;

![Tipo_Cartao](https://github.com/oGuiCorreia/EbacSql-/blob/main/Tipo_Cartao.png?raw=true)

**ps:** Temos 4 categorias de cartões

**Estimativa Salarial**
*Query:* Select distinct salario_anual from credito;

![Salario_anual](https://github.com/oGuiCorreia/EbacSql-/blob/main/Salario_anual.png?raw=true)

> A faixa salarial é categorizada em intervalos anuais. Também há registros com valores nulos.

## **ANALISE**

Agora como sabemos como o dataset estra estruturado e suas colunas podemos analisar suas informações.

**Vamos verificar qual a variação de idade de nosso usuarios**
*Query:* Select avg(idade) as media_idade, min(idade) as idade_minima , max(idade) as idade_maxima, tipo_cartao from credito;

![Variacao_Idade](https://github.com/oGuiCorreia/EbacSql-/blob/main/Variacao_Idade.png?raw=true)

>A idade dos clientes varia de 26 a 73 anos, com média de 45 anos.

**Quantos clientes temos de cada faixa salarial**
*Query:* Select count(sexo) as Quantidade_Usuarios, salario_anual from credito GROUP BY salario_anual;

![Quantidade_Usuario_Salario_anual](https://github.com/oGuiCorreia/EbacSql-/blob/main/Quantidade_Usuarios_Salario_Anual.png?raw=true)

>Podemos perceber que temos uma variação entre a faixa salarial, mas a maior quantidade de usuários tem o salario menor que $ 40K tendo no total 701 usuários 

**Qual é a quantidade de usuarios em cada tipo de cartão ?**
*Query:* Select count(sexo) as Quantidade_Usuarios, tipo_cartao from credito GROUP BY tipo_cartao;

![Quantidade_Usuario_Tipo_Cartão](https://github.com/oGuiCorreia/EbacSql-/blob/main/Quantidade_Usuarios_Tipo_Cartao.png?raw=true)

>A maioria dos clientes possui o cartão "Blue".

**Quais é o nosso limite minimo e o maximo no dataset?**
*Query:* Select min(limite_credito) as min_limite_credito, max(limite_credito) from credito limit 1;

![Limite_min_max_credito](https://github.com/oGuiCorreia/EbacSql-/blob/main/Min_max_limite_credito.png?raw=true)

**Maiores limites de credito**

*Query:* 
Select max(limite_credito) as max_limite_credito, salario_anual, tipo_cartao
from credito
where tipo_cartao != 'na' and salario_anual != 'na' 
GROUP BY salario_anual, tipo_cartao
order by max_limite_credito desc limit 10

![Max_limite_e_Descricao](https://github.com/oGuiCorreia/EbacSql-/blob/main/Max_credito_Descricao.png?raw=true)

>O maior limite de crédito pertence a um cliente com cartão "Blue". Apenas um cartão "Platinum" aparece no top 10.

**Quantidade de Produtos Comprados x Meses Inativos por Tipo de Cartão**

*Query:* 
select tipo_cartao, sum(qtd_produtos) as total_produtos_comprados, sum(meses_inativo_12m) as total_meses_inativos, count(*) as qnt_usuarios
from credito 
group by tipo_cartao
order by total_produtos_comprados desc

![Media_prod_comprados_Meses_inativos](https://github.com/oGuiCorreia/EbacSql-/blob/main/Media_prod_comprados_Meses_inativos.png?raw=true)

> Cartões "Blue" e "Silver" lideram em média de produtos comprados por usuário.

**Valor das Transações Anuais x Quantidade de Transações por Tipo de Cartão**

*Query:* 
select tipo_cartao, sum(valor_transacoes_12m ) as total_valor_transacoes, avg(valor_transacoes_12m ) as media_valor_transacoes, sum(qtd_transacoes_12m) as total_quantidade_transacoes, avg(qtd_transacoes_12m) as media_qnt_transacoes, count(*) as qnt_usuarios
from credito
GROUP BY tipo_cartao
order by total_quantidade_transacoes desc

![Valor_transacoes_e_Total_de_Transacoes](https://github.com/oGuiCorreia/EbacSql-/blob/main/Valor_transacoes_e_Total_de_Transacoes.png?raw=true)

> Embora cartões "Blue" e "Silver" dominem em volume de transações, o cartão "Gold" destaca-se pelo maior valor médio por transação.



## **CONCLUSÃO**

Essas foram algumas das analises possiveis com o **dataset**

**Insights:**

    > Podemos ver que a maior parte dos clientes tem renda menor que 40K
    > Temos cartões blue e silver dentre maiores limites de credito
    > Possuimos poucos cartoes platinum e um deles não aparece no top 10 de limite de credito
    > O cartão gold possui as melhores medias no quesito de valores e quantidade de transações
    > O cartão com o maior limite é um blue

**Uma otima opção para o banco em questão para melhoria seria uma análise mais profunda e detalhada sobre a oferta de cartões melhores para seus clientes. Isso se baseia no fato que metade de seus maiores limites de credito sao cartões blue(3) e silver(3). Assim poderia ser uma estratégia interessante para atrair clientes de maior potencial de crédito e atualizar os cartões de usuarios com limites altos**
